In [1]:
import os
import cv2
import h5py
import numpy as np
import scipy.spatial as spatial
from scipy.io import loadmat

In [2]:
# Adaptive Gaussian kernel
def get_density_map_gaussian(im, points, adaptive_mode=False, fixed_value=15, fixed_values=None):
    density_map = np.zeros(im.shape[:2], dtype=np.float32)
    h, w = density_map.shape[:2]
    num_gt = np.squeeze(points).shape[0]

    if num_gt == 0:
        return density_map

    if adaptive_mode == True:
        fixed_values = None
        leafsize = 2048
        tree = spatial.KDTree(points.copy(), leafsize=leafsize)
        distances, locations = tree.query(points, k=4)

    for idx, p in enumerate(points):
        p = np.round(p).astype(int)
        p[0], p[1] = min(h-1, p[1]), min(w-1, p[0])
        if num_gt > 1:
            if adaptive_mode == 1:
                sigma = int(np.sum(distances[idx][1:4]) * 0.1)
            elif adaptive_mode == 0:
                sigma = fixed_value
        else:
            sigma = fixed_value
        sigma = max(1, sigma)
        gaussian_radius_no_detection = sigma * 3
        gaussian_radius = gaussian_radius_no_detection

        if fixed_values is not None:
            grid_y, grid_x = int(p[0]//(h/3)), int(p[1]//(w/3))
            grid_idx = grid_y * 3 + grid_x
            gaussian_radius = fixed_values[grid_idx] if fixed_values[grid_idx] else gaussian_radius_no_detection
        gaussian_map = np.multiply(
            cv2.getGaussianKernel(gaussian_radius*2+1, sigma),
            cv2.getGaussianKernel(gaussian_radius*2+1, sigma).T
        )
        gaussian_map[gaussian_map < 0.0003] = 0
        if np.sum(gaussian_map):
            gaussian_map = gaussian_map / np.sum(gaussian_map)
        x_left, x_right, y_up, y_down = 0, gaussian_map.shape[1], 0, gaussian_map.shape[0]
        # cut the gaussian kernel
        if p[1] < gaussian_radius:
            x_left = gaussian_radius - p[1]
        if p[0] < gaussian_radius:
            y_up = gaussian_radius - p[0]
        if p[1] + gaussian_radius >= w:
            x_right = gaussian_map.shape[1] - (gaussian_radius + p[1] - w) - 1
        if p[0] + gaussian_radius >= h:
            y_down = gaussian_map.shape[0] - (gaussian_radius + p[0] - h) - 1
        density_map[
            max(0, p[0]-gaussian_radius):min(density_map.shape[0], p[0]+gaussian_radius+1),
            max(0, p[1]-gaussian_radius):min(density_map.shape[1], p[1]+gaussian_radius+1)
        ] += gaussian_map[y_up:y_down, x_left:x_right]
    # density_map[density_map < 0.0003] = 0
    density_map = density_map / (np.sum(density_map / num_gt))
    return density_map

In [3]:
# data root
original_data_root = 'data/original/ShanghaiTech'
processed_data_root = 'data/processed/ShanghaiTech'

print('start generating...')

for dataset in ['A', 'B']:
    dataset_path = os.path.join(original_data_root, 'part_{}'.format(dataset))
    is_adaptive = True if dataset is 'A' else False

    for data in ['test_data', 'train_data']:
        data_path = os.path.join(dataset_path, data)
        img_path = os.path.join(data_path, 'images')
        img_files = os.listdir(img_path)
        gt_path = os.path.join(data_path, 'ground-truth')
        gt_files = os.listdir(gt_path)
        # check num of files: gt = images ?
        if len(gt_files) != len(img_files):
            print('num error! please check dataset.')
            exit()

        # make dirs
        processed_gt_path = os.path.join(processed_data_root, dataset, data, 'ground-truth')
        if not os.path.exists(processed_gt_path):
            os.makedirs(processed_gt_path)

        # generate density map
        for i in range(1, len(img_files)+1):
            img = cv2.imread(os.path.join(img_path, 'IMG_'+str(i)+'.jpg'))
            m = loadmat(os.path.join(gt_path, 'GT_IMG_'+str(i)+'.mat'))
            gt = m['image_info'][0][0][0][0][0] - 1
            k = np.zeros(img.shape[0:2])
            for j in range(len(gt)):
                if int(gt[j][1]) < img.shape[0] and int(gt[j][0]) < img.shape[1]:
                    k[int(gt[j][1]), int(gt[j][0])]=1
            density = get_density_map_gaussian(k, gt, adaptive_mode=is_adaptive)
            
            # write density map into .h5 file
            with h5py.File(os.path.join(processed_gt_path, 'GT_IMG_'+str(i)+'.h5'), 'w') as f:
                f['density'] = density

print('done')

start generating...
done
